# Iris 꽃받침 / 꽃잎 너비 예측 경진대회 (DACON)
붓꽃의 종류, 꽃받침, 꽃잎의 길이를 이용해 꽃받침의 너비와 꽃잎의 너비를 예측해보세요.

In [194]:
import pandas as pd
import numpy as np

## 3. 모델링
-> 시각화 결과  
1) 데이터 개수가 많지않고  
2) 꽃받침과 꽃잎모두 길이에 따라 우상향 하는 모습이 보여짐.  
3) 꽃받침에서 setosa가 다른종과 확연히 구분되는 것으로 보여짐.           
4) 꽃잎에서는 꽃받침에 비해 종별로 분포가 확실히 구분되는 모습이 보여짐. (파->노->초)  


-----------------
종별로 분포 차이 보이므로 standard scaler ?

### 1) feature engineering

(1) 훈련세트 만들기 (id열 제거, 범주형데이터처리)

In [195]:
iris = pd.read_csv('iris_train.csv')
tr_input = iris.iloc[:,1:4]
tr_target = iris.iloc[:,4:6]

In [196]:
print(tr_input,tr_target)

       species  sepal length (cm)  petal length (cm)
0       setosa                4.4                1.4
1   versicolor                6.4                4.5
2    virginica                6.2                4.8
3    virginica                7.2                6.1
4       setosa                4.9                1.4
..         ...                ...                ...
70  versicolor                6.5                4.6
71  versicolor                5.6                3.6
72  versicolor                6.2                4.5
73  versicolor                4.9                3.3
74  versicolor                6.9                4.9

[75 rows x 3 columns]     sepal width (cm)  petal width (cm)
0                2.9               0.2
1                3.2               1.5
2                2.8               1.8
3                3.6               2.5
4                3.0               0.2
..               ...               ...
70               2.8               1.5
71               2.9         

In [197]:
def set(species):
    if species == 'setosa':
        return 1
    else :
        return 0
    
def ver(species):
    if species == 'versicolor':
        return 1
    else :
        return 0   
    
def vir(species):
    if species == 'virginica':
        return 1
    else :
        return 0        

In [198]:
species = tr_input['species']
tr_input['setosa'] = species.apply(set)
tr_input['versicolor'] = species.apply(ver)
tr_input['virginica'] = species.apply(vir)
del tr_input['species']

In [199]:
tr_input.head() # 훈련세트 완성
tr_sep = tr_input.drop(['petal length (cm)'],axis=1)
tr_pet = tr_input.drop(['sepal length (cm)'],axis=1)
tr_target_sep = tr_target['sepal width (cm)']
tr_target_pet = tr_target['petal width (cm)']

In [200]:
tr_pet

,petal length (cm),setosa,versicolor,virginica
0,1.4,1,0,0
1,4.5,0,1,0
2,4.8,0,0,1
3,6.1,0,0,1
4,1.4,1,0,0
...,...,...,...,...
70,4.6,0,1,0
71,3.6,0,1,0
72,4.5,0,1,0
73,3.3,0,1,0


In [201]:
print(tr_target_sep,tr_target_pet)

0     2.9
1     3.2
2     2.8
3     3.6
4     3.0
     ... 
70    2.8
71    2.9
72    2.2
73    2.4
74    3.1
Name: sepal width (cm), Length: 75, dtype: float64 0     0.2
1     1.5
2     1.8
3     2.5
4     0.2
     ... 
70    1.5
71    1.3
72    1.5
73    1.0
74    1.5
Name: petal width (cm), Length: 75, dtype: float64


## 2) 모델 적합하기

In [202]:
# 평가지표

# def MAE(true, pred):
#     score = np.mean(np.abs(true-pred))
#     return score

## 1. 선형회귀 (단변량회귀로 y 나누어서 추정)

### (1) 단순선형회귀

In [203]:
# tr1_sep = tr_input[species == 'setosa']['sepal length (cm)']
# tr2_sep = tr_input[species == 'versicolor']['sepal length (cm)']
# tr3_sep = tr_input[species == 'virginica']['sepal length (cm)']

In [204]:
# tr1_pet = tr_input[species == 'setosa']['sepal length (cm)']
# tr2_pet = tr_input[species == 'versicolor']['sepal length (cm)']
# tr3_pet = tr_input[species == 'virginica']['sepal length (cm)']

### (2) 다중선형회귀 

In [205]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(include_bias=False)

# poly fit
poly.fit(tr_sep)
poly.fit(tr_pet)

sep_poly = poly.transform(tr_sep)
pet_poly = poly.transform(tr_pet)

* (+) k-폴드 교차검증  

    *  정답 데이터가 없으므로 5-폴드 교차 검증으로 평가
    

* 교차검증이란 ?
    검증세트를 떼어내어 평가하는 과정을 여러번 반복하고 이 점수를 평균하여 최종 검증 점수를 얻음. 
         
         from sklearn.model_selsection import cross_validate
         scores = cross_validate(평가모델,train_input,train_target)
         scores

* score의 결과로 fit_time, score_time,test score 키를 가진 딕셔너리를 반환
* 교차검증의 점수는 test_Score에 담긴 5개 값들의 평균으로 구함


In [206]:
# sepal width 교차검증
from sklearn.model_selection import cross_validate

scores = cross_validate(lr,sep_poly, tr_target_sep,cv=5,scoring='neg_mean_absolute_error')
scores
# 사이킷런은 Neg MAE만 제공

{'fit_time': array([0.00498557, 0.00199509, 0.00199509, 0.00199533, 0.00199127]),
 'score_time': array([0.00099659, 0.00099587, 0.00099659, 0.00099754, 0.        ]),
 'test_score': array([-0.20208687, -0.25175582, -0.1816483 , -0.21892794, -0.17866247])}

In [207]:
np.mean(scores['test_score'])*(-1)

0.20661628000379761

In [208]:
# petal width 교차검증

scores = cross_validate(lr,pet_poly, tr_target_pet,cv=5,scoring='neg_mean_absolute_error')
scores

{'fit_time': array([0.01198602, 0.00199533, 0.00299168, 0.00400805, 0.00498939]),
 'score_time': array([0.00197339, 0.00502872, 0.00099707, 0.00097823, 0.0009973 ]),
 'test_score': array([-0.10430386, -0.14669654, -0.16863147, -0.14616677, -0.14441042])}

In [209]:
np.mean(scores['test_score'])*(-1)

0.14204181071578806

In [210]:
poly.get_feature_names() # pet 

['x0',
 'x1',
 'x2',
 'x3',
 'x0^2',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x1^2',
 'x1 x2',
 'x1 x3',
 'x2^2',
 'x2 x3',
 'x3^2']

ㄴ>  1) petal과 sepal width간 score차이가 매우 큼.  
ㄴ>  2) 범주형 변환하면서 추가된 데이터들로 의미없는 변수 생김 

__________

In [211]:
### poly 없이 sepal width 교차검증

socres = cross_validate(lr,tr_sep, tr_target_sep,cv=5,scoring='neg_mean_absolute_error')
scores

{'fit_time': array([0.01198602, 0.00199533, 0.00299168, 0.00400805, 0.00498939]),
 'score_time': array([0.00197339, 0.00502872, 0.00099707, 0.00097823, 0.0009973 ]),
 'test_score': array([-0.10430386, -0.14669654, -0.16863147, -0.14616677, -0.14441042])}

In [212]:
np.mean(scores['test_score'])*(-1) # poly 적용시 0.20661628000379761 

# poly 적용 결과가 더 좋음

0.14204181071578806

In [213]:
### poly 없이 petal width 교차검증

scores = cross_validate(lr,tr_pet, tr_target_pet,cv=5,scoring='neg_mean_absolute_error')
scores

{'fit_time': array([0.01097178, 0.00896907, 0.00398684, 0.00498652, 0.00299168]),
 'score_time': array([0.00797892, 0.00199628, 0.00199461, 0.00199509, 0.00199485]),
 'test_score': array([-0.10784067, -0.13186018, -0.17402951, -0.13456578, -0.14259946])}

In [214]:
np.mean(scores['test_score'])*(-1) # poly 적용시 0.14204181071578806 

# poㅣy 미적용시 결과가 더 좋음.

0.1381791211512438

## 2. 앙상블 -  XGBoost 


#### XGBoost
- XGBoost는 자체적으로 정규화 파라미터 제공(gamma, alpha, lambda)
- XGBoost 파라미터
  - learning_rate : 오버피팅을 막기위해 사용되는 축소 단계 크기로 0과 1사이의 값이다.
  - max_depth : 부스팅 라운드 동안 각각의 트리가 얼마나 깊게 성장할 수 있는지 결정
  - subsample : 트리당 사용되는 샘플의 비율. 낮은 값은 언더피팅으로 이어질 수 있다.
  - colsample_bytree : 트리당 사용되는 특성의 비율. 높은 값은 오버피팅으로 이저질 수 있다.
  - n_estimators : 구축하고자하는 트리의 개수
  - objective : 손실함수 결정
  - reg:linear : 회귀문제
  - reg:logistic : 오직 결정(선택)만 있는 분류 문제
  - binary:logistic : 확률을 포함하는 분류 문제
- Feature Importance

In [215]:
!pip install xgboost

In [216]:
import xgboost as xgb
from xgboost import XGBRegressor, plot_tree, plot_importance

reg = XGBRegressor(max_depth = 3, n_estimators = 100)

In [217]:
# 교차검증
scores = cross_validate(reg,tr_input,tr_target_sep,scoring='neg_mean_absolute_error')
scores

{'fit_time': array([0.14860058, 0.13862658, 0.14062381, 0.14361477, 0.14760327]),
 'score_time': array([0.01197028, 0.01396346, 0.01296616, 0.01296473, 0.01396275]),
 'test_score': array([-0.34122555, -0.31523401, -0.32876084, -0.26012327, -0.30825504])}

In [218]:
np.mean(scores['test_score'])*(-1) #0.3107197405497233

0.3107197405497233

## 3) 하이퍼파라미터 튜닝

### CV후 교차검증 - sepal width

In [219]:
from sklearn.model_selection import GridSearchCV
reg_cv = XGBRegressor()
xgb_params = {'max_depth' : [3,4, 5, 6],
              'n_estimators' : [10, 20, 50, 100, 500,1000],
              'learning_rate' : [0.1, 0.2, 0.4, 0.6, 0.8, 1]}
grid = GridSearchCV(estimator = reg_cv, param_grid = xgb_params, scoring = 'neg_mean_absolute_error')

grid.fit(tr_sep, tr_target_sep)
best_param = grid.best_params_
print(best_param)

{'learning_rate': 0.4, 'max_depth': 3, 'n_estimators': 10}


In [220]:
reg = XGBRegressor(max_depth = 3, n_estimators = 10,learning_rate = 0.4 )

In [221]:
scores = cross_validate(reg,tr_sep,tr_target_sep,scoring='neg_mean_absolute_error')
scores

{'fit_time': array([0.034899  , 0.02991891, 0.03091526, 0.02991915, 0.031914  ]),
 'score_time': array([0.01396465, 0.01296449, 0.01296759, 0.01297593, 0.01297164]),
 'test_score': array([-0.25707975, -0.29209839, -0.2099418 , -0.22838141, -0.23353546])}

In [222]:
np.mean(scores['test_score'])*(-1) #0.244207

0.2442073631286621

In [223]:
## feature별 중요도 파악 - 시각화
# import matplotlib.pyplot as 
# pltreg.fit(train_input, train_target)
# fig, ax = plt.subplots(figsize=(9,10))
# plot_importance(reg, ax=ax)

### CV후 교차검증 - petal width

In [224]:
from sklearn.model_selection import GridSearchCV
reg_cv = XGBRegressor()
xgb_params = {'max_depth' : [3,4, 5, 6],
              'n_estimators' : [10, 20, 50, 100, 500,1000],
              'learning_rate' : [0.1, 0.2, 0.4, 0.6, 0.8, 1]}
grid = GridSearchCV(estimator = reg_cv, param_grid = xgb_params, scoring = 'neg_mean_absolute_error')

grid.fit(tr_pet, tr_target_pet)
best_param = grid.best_params_
print(best_param)

{'learning_rate': 0.4, 'max_depth': 3, 'n_estimators': 10}


In [225]:
reg = XGBRegressor(max_depth = 3, n_estimators = 10,learning_rate = 0.4 )
scores = cross_validate(reg,tr_pet, tr_target_pet,scoring='neg_mean_absolute_error')
scores

{'fit_time': array([0.04388213, 0.03091717, 0.03291202, 0.0309155 , 0.03191304]),
 'score_time': array([0.01296306, 0.01296735, 0.01296735, 0.01296639, 0.01396513]),
 'test_score': array([-0.11942495, -0.19776278, -0.16259066, -0.1425414 , -0.15718262])}

In [226]:
np.mean(scores['test_score'])*(-1) #0.1559004

0.15590047907829283